In [495]:
import sys
import os
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, Input, Output,State,ctx # Required to detect which button was clicked
import dash_bootstrap_components as dbc
import plotly.express as px
sys.path.append(os.path.abspath(".."))

In [496]:
df= pd.read_csv('../data/bird_migration.csv')

df.head()

,Unnamed: 0,altitude,date_time,device_info_serial,direction,latitude,longitude,speed_2d,bird_name
0,0,71,2013-08-15 00:18:08+00,851,-150.469753,49.419859,2.120733,0.150000,Eric
1,1,68,2013-08-15 00:48:07+00,851,-136.151141,49.419880,2.120746,2.438360,Eric
2,2,68,2013-08-15 01:17:58+00,851,160.797477,49.420310,2.120885,0.596657,Eric
3,3,73,2013-08-15 01:47:51+00,851,32.769360,49.420359,2.120859,0.310161,Eric
4,4,69,2013-08-15 02:17:42+00,851,45.191230,49.420331,2.120887,0.193132,Eric


In [497]:
# --- 4. MAP VISUALIZATION ENGINE ---
def create_map(df):
    if df.empty:
        fig = px.scatter_geo()
        fig.update_layout(template="plotly_white", paper_bgcolor="rgba(0,0,0,0)")
        fig.add_annotation(text="No data selected", x=0.5, y=0.5, showarrow=False)
        return fig
    
    # Transform data for Line + Marker plotting
    plot_data = []
    for index, row in df.iterrows():
        segment_id = f"{row['bird_name']}_{index}"
        
        # # Get the Reason, Remove Coordinate Strings ---
        # # Ensure your column name matches 'Migration_Reason'
        # reason = row['Migration_Reason'] 
        
        # Start Point
        plot_data.append({
            "Bird_name": row['bird_name'],
            "Latitude": row['latitude'],
            "Longitude": row['longitude'],
            "Segment_ID": segment_id
        })
    
    df_plot = pd.DataFrame(plot_data)


    # Plot
    fig = px.line_geo(
        df_plot,
        lat="Latitude", lon="Longitude", color="Bird_name",
        line_group="Bird_name", 
        hover_name="Bird_name", 
        
        #  Update Hover Data
        hover_data={
            # "bird_name": True, 
            # "Migration Reason": True, 
            # "Position": True, 
            "Bird_name": True, 
            "Latitude": False, 
            "Longitude": False
        },
        
        projection="orthographic", 
        title=f"Tracking {df['bird_name'].nunique()} Unique Birds",
        color_discrete_sequence=px.colors.qualitative.Bold,
        fitbounds="locations"
    )

    # Styling: Lines + Markers
    fig.update_traces(
        mode='lines+markers', 
        line=dict(width=2), 
        marker=dict(size=6, symbol='circle', opacity=1, line=dict(width=0)),
        opacity=0.8
    )
    
    # Map Geos styling
    fig.update_geos(
        visible=True, resolution=50,
        showcountries=True, countrycolor="#bbbbbb",
        showcoastlines=True, coastlinecolor="#bbbbbb",
        showland=True, landcolor="#f0f0f0",      
        showocean=True, oceancolor="#e4edff",   
        projection_rotation=dict(lon=-10, lat=20)
    )
    
    fig.update_layout(
        template="plotly_white",
        margin={"r":0,"t":50,"l":0,"b":0},
        paper_bgcolor="rgba(0,0,0,0)", 
        legend=dict(yanchor="top", y=0.95, xanchor="left", x=0.05, bgcolor="rgba(255,255,255,0.9)")
    )
    return fig

# create_map(df).show()

In [498]:
df

,Unnamed: 0,altitude,date_time,device_info_serial,direction,latitude,longitude,speed_2d,bird_name
0,0,71,2013-08-15 00:18:08+00,851,-150.469753,49.419859,2.120733,0.150000,Eric
1,1,68,2013-08-15 00:48:07+00,851,-136.151141,49.419880,2.120746,2.438360,Eric
2,2,68,2013-08-15 01:17:58+00,851,160.797477,49.420310,2.120885,0.596657,Eric
3,3,73,2013-08-15 01:47:51+00,851,32.769360,49.420359,2.120859,0.310161,Eric
4,4,69,2013-08-15 02:17:42+00,851,45.191230,49.420331,2.120887,0.193132,Eric
...,...,...,...,...,...,...,...,...,...
61915,61915,11,2014-04-30 22:00:08+00,833,45.448157,51.352572,3.177151,0.208087,Sanne
61916,61916,6,2014-04-30 22:29:57+00,833,-112.073055,51.352585,3.177144,1.522662,Sanne
61917,61917,5,2014-04-30 22:59:52+00,833,69.989037,51.352622,3.177257,3.120545,Sanne
61918,61918,16,2014-04-30 23:29:43+00,833,88.376373,51.354641,3.181509,0.592115,Sanne


In [499]:
# Aggregate Data
df_bird_stats = df.groupby('bird_name').agg(
    Max_Altitude=('altitude', 'max'),
    Avg_Altitude=('altitude', 'mean'),
    Min_Altitude=('altitude', 'min'),
    Max_Speed=('speed_2d', 'max'),
    Avg_Speed=('speed_2d', 'mean'),
    Min_Speed=('speed_2d', 'min')
).reset_index()

In [500]:
# Bar chart
def build_bar_chart(df_bird_stats, selected_bird, category, selected_stats):

    # Empty State
    if not selected_bird:
        fig = px.bar()
        fig.update_layout(
            title={
                'text': "Select bird from the configuration panel to begin analysis",
                'y': 0.5, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'middle'
            },
            xaxis={'visible': False}, yaxis={'visible': False},
            plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)',
            font=dict(size=18, color="#6c757d")
        )
        return fig

    # Setup Columns and Labels based on Category
    cols_to_plot = []

    if category == "Altitude":
        y_label = "Altitude (meters)"
        title_prefix = "Altitude Comparison"

        color_map = {
            'Max_Altitude': '#0d6efd',
            'Avg_Altitude': '#aecdf7',
            'Min_Altitude': '#fd7e14'
        }

        # Build Column List
        for stat in selected_stats:
            cols_to_plot.append(f"{stat}_{category}")

    else:  # Speed
        y_label = "Speed"
        title_prefix = "Speed Comparison"

        color_map = {
            'Max_Speed': '#198754',
            'Avg_Speed': '#a3cfbb',
            'Min_Speed': '#fd7e14'
        }
        
        # Build Column List
        for stat in selected_stats:
            cols_to_plot.append(f"{stat}_{category}")

    # Filter Data
    df_filtered = df_bird_stats[df_bird_stats['bird_name'].isin(selected_bird)]

    # Melt Data
    valid_cols = [c for c in cols_to_plot if c in df_filtered.columns]

    df_melted = df_filtered.melt(
        id_vars='bird_name',
        value_vars=valid_cols,
        var_name='Metric',
        value_name='Value'
    )

    if df_melted.empty:
        return px.bar(title="No data available for the selected options.")

    # Build Figure
    fig = px.bar(
        df_melted,
        x='bird_name',
        y='Value',
        color='Metric',
        barmode='group',
        title=title_prefix,
        template='plotly_white',
        text_auto=',.0f',
        color_discrete_map=color_map
    )

    fig.update_layout(
        legend_title_text="Metric",
        xaxis_title=None,
        margin=dict(t=60, b=40, l=40, r=180),
        legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02),
        yaxis=dict(title=y_label, tickformat=","),
        transition={'duration': 500}
    )

    return fig

In [501]:

# Filter Data
df_filtered = df_bird_stats[df_bird_stats['bird_name'].isin(["Eric"])]
df_filtered.head()


# Melt Data
valid_cols = [c for c in ['Average_Altitude'] if c in df_filtered.columns]

df_melted = df_filtered.melt(
    id_vars='bird_name',
    value_vars=valid_cols,
    var_name='Metric',
    value_name='Value'
)
df_melted.head()

,bird_name,Metric,Value


# Application Inicialization

In [502]:

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server


# Dashboard Layout

In [503]:
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H2("Global Bird Migration Tracker", className="display-6"), width=12),
        dbc.Col(html.P("Compare bird_name between specific bird IDs.", className="text-muted"), width=12),
    ], className="my-4"),
    
    dbc.Row([
        # ------ SIDEBAR -------
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Filter Controls", className="fw-bold"),
                dbc.CardBody([
                    #bird_name SECTION 
                    html.Label("Select Bird", className="mb-2 fw-bold text-primary"),
                    dcc.Dropdown(
                        id='bird-name-filter',
                        options=[{'label': s, 'value': s} for s in sorted(df['bird_name'].unique())],
                        value=sorted(df['bird_name'].unique()), 
                        multi=True, 
                        clearable=True
                    ),
                    #Button to select all bird_name
                    dbc.Button("Select All Birds", id="btn-all-birds", color="light", size="sm", className="mt-1 w-100 border"),

                    html.Hr(),
                    
                    # DATA CATEGORY
                    html.Label("Choose Data Category", className="fw-bold text-primary"),
                    dbc.RadioItems(
                        id='category-selector',
                        options=[
                            {'label': ' Altitude', 'value': 'Altitude'},
                            {'label': ' Speed', 'value': 'Speed'}
                        ],
                        value='Altitude', 
                        className="mb-3",
                        inputClassName="me-2"
                    ),

                    html.Hr(),

                    # STATISTIC CHECKLIST
                    html.Label("Select Statistics", className="fw-bold text-primary"),
                    dbc.Checklist(
                        id='statistic-checklist',
                        options=[
                            {'label': ' Maximum', 'value': 'Max'},
                            {'label': ' Average', 'value': 'Avg'},
                            {'label': ' Minimum', 'value': 'Min'},
                            # Real World is added dynamically, but we can default it here too
                        ],
                        value=['Max', 'Avg', 'Min'], 
                        switch=True, 
                        className="mb-3"
                    ),
                ])
            ], className="mb-4 shadow-sm")
        ], width=12, md=3), 
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='migration-map', style={'height': '75vh'}) 
                ], style={'padding': '0'})
            ], className="shadow-sm")
        ], width=12, md=9),
        
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    dcc.Graph(id='main-bar-chart', style={'height': '75vh'}) 
                ], style={'padding': '0'})
            ], className="shadow-sm border-0")
        ], width=12, md=9)
    ]),
], fluid=True)

# Interactivity & Logic (Callbacks)

In [504]:
@app.callback(
    Output('migration-map', 'figure'),
    Input('bird-name-filter', 'value')
)
def update_map(selected_bird_names):

    # When nothing is selected → return empty map
    if not selected_bird_names:
        return create_map(pd.DataFrame())

    # Filter by bird_name (since that's the dropdown value)
    filtered = df[df['bird_name'].isin(selected_bird_names)]

    return create_map(filtered)

# Callback: Update Options (Real World is now available for BOTH)
@app.callback(
    [Output('statistic-checklist', 'options'),
     Output('statistic-checklist', 'value')],
    Input('category-selector', 'value'),
    State('statistic-checklist', 'value')
)
def update_stat_options(category, current_values):
    options = [
        {'label': ' Maximum', 'value': 'Max'},
        {'label': ' Average', 'value': 'Avg'},
        {'label': ' Minimun', 'value': 'Min'}
    ]

    return options, current_values


@app.callback(
    Output('main-bar-chart', 'figure'),
    [
        Input('bird-name-filter', 'value'),
        Input('category-selector', 'value'),
        Input('statistic-checklist', 'value')
    ]
)
def update_chart(selected_bird, category, selected_stats):
    return build_bar_chart(
        df_bird_stats=df_bird_stats,
        selected_bird=selected_bird,
        category=category,
        selected_stats=selected_stats
    )

@app.callback(
    Output('bird-name-filter', 'value'),
    Input('btn-all-birds', 'n_clicks'),
    State('bird-name-filter', 'options'),
    prevent_initial_call=True
)
def select_all_species(n_clicks, options):
    return [opt['value'] for opt in options]



# Application Execution

In [505]:
if __name__ == '__main__':
    app.run(debug=True)